# ONE Quick Start
This tutorial will get you started searching and loading IBL data using Open Neurophysiology
Environment (ONE).

First we need to install ONE. If you don't already have IBL libraries, the easiest way is to run
 `pip install ONE-api`.

Now we need to import the ONE library and open a connection to the IBL public data server. To do
so, we create an ONE object, and ask it to connect to the IBL public server.

<div class="alert alert-info">
Info.

IBL internal users may use their Alyx credentials to access all IBL data.
[Click here](https://int-brain-lab.github.io/ONE/one_installation.html#connecting-to-specific-database-relevant-for-ibl-users) for details.
</div>

In [ ]:
from one.api import ONE
one = ONE(base_url='https://openalyx.internationalbrainlab.org', password='international', silent=True)

Now we are going to search for an experiment to analyze. First let's find out what we can search by:

In [ ]:
print(one.search_terms())

Let's search for sessions recorded in August 2020, which contain a dataset 'probes.description',
meaning that electrophysiology was recorded. By adding the argument `details=True`, we get two
outputs - the experiment IDs uniquely identifying these sessions, and some information about the
experiments.

In [ ]:
eids, info = one.search(date_range=['2020-08-01', '2020-08-31'], dataset='probes.description', details=True)

from pprint import pprint
pprint(eids)
pprint(info)

So there were four experiments matching the criteria in the public database. Now let's load the
probe information for the first experiment. `one.load_dataset` returns a single named dataset

In [ ]:
eid = eids[0]  # Select the first experiment
probe_insertions = one.load_dataset(eid, 'probes.description')

print(f'N probes = {len(probe_insertions)}')
pprint(probe_insertions[0])

Now let’s see all the datasets associated with the first of these experiments. The command one.list_datasets returns the full path of all datasets, including the collection name and the extension. The ‘alf’ collection contains the preprocessed data we usually want to work with, and the data for each probe are in labeled subdirectories. We use the wildcard * because the data can be saved in different subdirectories for different spike sorters.

In [ ]:
probe_label = probe_insertions[0]['label']
one.list_datasets(eid, collection=f'alf/{probe_label}*')

We might be interested in the data of this session that is not specific to the probe recording, e.g. the behavioural data or video data. We can find that if we list datasets in the alf collection without specifying the probe.

In [ ]:
one.list_datasets(eid, collection='alf')

Let's load the preprocessed data associated with the left camera. There are two ways to do this.
`one.load_dataset` or `one.load_object`, which returns all the datasets with the same name part,
as an object. Let's use the second method to load all left camera data in the alf folder.

In [ ]:
cam = one.load_object(eids[0], 'leftCamera', collection='alf')
cam.keys()

To load only specific data associated with this object, we can use the attribute keyword. Let's load the times and the DLC traces.

In [ ]:
cam = one.load_object(eids[0], 'leftCamera', collection='alf', attribute=['times', 'dlc'])
cam.keys()

And that's the end of the quick start tutorial! For more information on any of these commands you
can use the standard help function:

In [ ]:
help(one.list_datasets)

For detailed tutorials, guides and examples, [click here for the full ONE API documentation
Website](https://int-brain-lab.github.io/ONE/).